In [0]:
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
data_downloaded = drive.CreateFile({'id': '10yPw1zFzLAHIlQVSFPThPmbpA3Xv8zbY'})
data_downloaded.GetContentFile('train.csv')
data_downloaded = drive.CreateFile({'id': '1KSLxZ6Twuy50YxFoSos02nFsFNxPWj31'})
data_downloaded.GetContentFile('test.csv')

In [85]:
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
import re, nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
nltk.download('stopwords')
from  nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer 
import gensim
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
train =pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
def processTweet(tweet):
  re.sub("[^a-zA-Z]", " ", tweet)
  #Convert to lower case
  tweet = tweet.lower()
  #delete www.* or https?://* 
  tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
  #delete @username 
  tweet = re.sub('@[^\s]+','',tweet)
  #Remove additional white spaces
  tweet = re.sub('[\s]+', ' ', tweet)
  #Replace #word with word
  tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
  #trim
  tweet = tweet.strip('\'"')
  tweet =tweet.translate(str.maketrans(' ', ' ', string.punctuation))
  #removing the stop words using nltk
  stop_words = set(stopwords.words('english')) 
  tweet=word_tokenize(tweet)
  #make a copy of the word_list
  tweet = [w for w in tweet if not w in stop_words]
  ps = PorterStemmer()
  tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
  tweet = ' '.join(tweet)
  return tweet

In [0]:
train.TweetText=train.TweetText.apply(lambda line: processTweet(line))
test.TweetText=test.TweetText.apply(lambda line: processTweet(line))

In [89]:
corpus= pd.concat([train,test],axis = 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [97]:
tokenized_tweet = corpus['TweetText'].apply(lambda x: x.split()) 
model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=220, # desired no. of features/independent variables
            window=6, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2,# no.of cores
            seed = 34
            
            ) 

model_w2v.train(tokenized_tweet, total_examples= len(corpus['TweetText']), epochs=27) 

(2124747, 2374623)

In [0]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size)) 
            count += 1.
        except KeyError:                                   
            continue
    if count != 0:
        vec /= count
    return vec

In [98]:
import numpy as np
wordvec_arrays = np.zeros((len(tokenized_tweet), 220)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet.iloc[i], 220)
    wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(9135, 220)

In [0]:
train_features_wordvec = wordvec_df.iloc[:6525,:]

test_features_wordvec = wordvec_df.iloc[6525:,:]

In [0]:
y=train['Label']

In [101]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf')
classifier.fit(train_features_wordvec, y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
prediction=classifier.predict(test_features_wordvec)

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
s1 =pd.Series(prediction)
result1 = s1.rename('Label') 
df= test.join(result1)
df = df.drop('TweetText', axis=1)
df.to_csv('submission5.csv',index=False)
!cp submission5.csv drive/My\ Drive/